In [1]:
from typing import List

from tictactoe import State, Result, Player, playout, Command, other_player, pick_move_by_playouts
from math import sqrt, log
import random
import time
import numpy as np

'root' is current game state

'leaf' is any node that has a potential child from which no playout has been run.

### Data structure

We build up a tree. 
Each node has:
* a game state
* possibly some child states
* a current ratio of wins *for the current state's player* versus total playouts down this path

### Selection
So start at the root and choose a leaf. 

(How we do this is all about the balance between exploration and exploitation) 
Building this may be the hard bit.

### Expansion

if the leaf state is not a finished game:
* pick a valid command
* create a new child state by applying that command

### Simulation
* do a playout from this state (in other systems this is replaced be e.g. neural net driven eval function)


### Back propagation
Update the win/loss ratio in this node and all parent nodes.

(This presumably means knowledge of 'parent' somehow in the data structure in addition to 'children')

In [2]:

class MCTSException(Exception):
    def __init__(self, node):
        self.node=node
        
        
class Node:
    'State is immutable, nodes are not'
    def __init__(self,state,parent=None, command=None):
        assert parent is not self
        if parent is None: 
            assert command is None

        self.command=command # the command that got us here, might be useful
        self.parent=parent
        self.state=state
        # we may want this to be a set?
        self.children:List[Node]=[]
        self.playouts=0
        self.wins=0
        
        self.depth = 0 if self.parent is None else 1+self.parent.depth

    def __repr__(self):
        ratio=self.ratio
        if ratio is not None:
            ratio=round(ratio,2)
        return f"Node(depth={self.depth},wins={self.wins},playouts={self.playouts}, ratio={ratio}, command={self.command})"
    
    @property
    def is_leaf(self):
        "'leaf' is any node that has a potential child from which no playout has been run."
        if len(self.children)<len(self.state.commands):
            return True
        if any(node.playouts==0 for node in self.children):
            assert 0, "Does this ever happen?"
            return True
        return False

    def size(self):
        return 1 + sum(child.size() for child in self.children)
        
    @property
    def ratio(self)->float:
        if self.wins==0:return 0
        return self.wins/self.playouts
    

    def tree(self):
        return {
            "command":self.command,
            "playouts":self.playouts,
            "wins":self.wins,
            "children":[child.tree() for child in self.children]
        }
def select(node:Node)->Node:
    '''
    given a node return a node that is a leaf.
    Implement explore vs exploit here!
    
    If this node is a leaf we return it, otherwise we pick a child 
    and call select on it
    '''
    if node.is_leaf:
        return node
    
    # ok, I'm not a leaf. Therefore the number of 
    # children I have is equal to the number of legal moves
    assert len(node.children)==len(node.state.commands)

    
    # we can't go down any further, but this node can't be expanded.
    if len(node.state.commands)==0:
        assert node.state.result!=Result.INPROGRESS
        assert len(node.children)==0
        return node
    
    
    for child in node.children:
        assert child.parent is node
    
    scores=[uct_score(child) for child in node.children]
    index=scores.index(max(scores))
    
    return select(node.children[index])
    
def uct_score(node):
    
    #choose in each node of the game tree the move for which the expression UCT has the highest value
    s_i = node.playouts
    s_p = node.parent.playouts
    assert node.parent is not None, "UCT requires node has a parent"
    assert s_p>0, "How come parent hasn't had any playouts?"
    w_i = node.wins
    c = sqrt(2) # exploration parameter
    
    #https://medium.com/@quasimik/monte-carlo-tree-search-applied-to-letterpress-34f41c86e238
    return w_i/s_i + c * sqrt(log(s_p)/s_i)
    

def expand(node)->Node:
    'create a new child state from this node'
    
    if node.state.result!=Result.INPROGRESS:
        # can't really expand any more, so we'll just
        # backprop the value of the finished game
        return node
    
    
    assert len(node.state.commands)>0
    assert node.is_leaf
    assert len(node.children)< len(node.state.commands)
    
    # pick a command that hasn't been used already
    possible=set(node.state.commands)              # available legal commands
    used=set(child.command for child in node.children) # commands that we've already expanded into the tree
    
    available = list(possible-used)
    command = random.choice(available)
    
    child = Node(
        state = node.state.applyCommand(command),
        parent=node,
        command=command
    )
    node.children.append(child)
    
    return child
    
def backprop(node,result):
    
    """ node.state.player is the player to play, so the player who has just played the move
    is the one we care about
    
    """
    assert result in (Result.PLAYER1,Result.PLAYER2,Result.DRAW)
    
    # the player who just played the move that got us to this state
    player = other_player(node.state.player)
    
    if (result==Result.PLAYER1 and player==Player.ONE):
        node.wins+=1
        
    if (result==Result.PLAYER2 and player==Player.TWO):
        node.wins+=1
    
    if (result==Result.DRAW):
        node.wins+=0.5
    
    node.playouts+=1

    if node.parent is not None:
        backprop(node.parent, result)
        
def mcts(root):
    leaf=select(root)
    c=expand(leaf)
    result =playout(c.state)
    backprop(c,result)

In [3]:
def best(root):
    '''
    Wikipedia says 
    "the move with the most simulations made (i.e. the highest denominator) is chosen as the final answer."
    
    Is that right?  do we want the best scoring move, or the most explored?
    '''
    if len(root.children)==0:
        return None # we haven't explored any commands from this node
    return max(root.children,key = lambda n:n.playouts).command


The REPL will work best if we can re-use already explored nodes.

In [4]:
def get_path(root):
    "pick a random path for analysis"
    path=[]
    path.append(root)
    while root.children:
        root = random.choice(root.children)
        path.append(root)   
    return path

In [5]:

def show(node):
    print(node)
    print(node.state)
    print(node.state.result)
    print(f"Player is {node.state.player}, best command is {best(node)} ")
    print('========='*8)

In [6]:
root=Node(State())
for i in range(1000):
    mcts(root)

In [7]:
path=get_path(root)
for node in path:
    show(node)

Node(depth=0,wins=333.0,playouts=1000, ratio=0.33, command=None)
 | | 
-----
 | | 
-----
 | |     player ONE
Result.INPROGRESS
Player is ONE, best command is Command(j=1, i=1) 
Node(depth=1,wins=34.0,playouts=63, ratio=0.54, command=Command(j=0, i=2))
 | |O
-----
 | | 
-----
 | |     player TWO
Result.INPROGRESS
Player is TWO, best command is Command(j=2, i=0) 
Node(depth=2,wins=5.5,playouts=10, ratio=0.55, command=Command(j=0, i=0))
X| |O
-----
 | | 
-----
 | |     player ONE
Result.INPROGRESS
Player is ONE, best command is Command(j=0, i=1) 
Node(depth=3,wins=0,playouts=1, ratio=0, command=Command(j=2, i=2))
X| |O
-----
 | | 
-----
 | |O    player TWO
Result.INPROGRESS
Player is TWO, best command is None 


---
### Compare MCTS with random playouts

In [8]:
s=State(m=np.array([
    [1,0,-1],
    [0,1,1],
    [0,0,-1],
]), player=Player.TWO)
s

O| |X
-----
 |O|O
-----
 | |X    player TWO

In [9]:
pick_move_by_playouts(s,1)

Command(j=2, i=1)

In [10]:
def pick_move_by_mcts(root,seconds):
    assert isinstance(root, Node)
    end=time.time()+seconds
    while time.time()<end:
        mcts(root)
    return best(root)
    
    

In [11]:
root=Node(s)

In [12]:
root.size()

1

In [13]:
root

Node(depth=0,wins=0,playouts=0, ratio=0, command=None)

In [14]:
pick_move_by_mcts(root, 1)

Command(j=1, i=0)

In [15]:
root

Node(depth=0,wins=12099.0,playouts=23810, ratio=0.51, command=None)

In [16]:
root.size()

50

Which is exactly what we're expecting: it assumes good play on the part of the opponent, not random play.

In [18]:
root

Node(depth=0,wins=12406.0,playouts=24423, command=None)

In [20]:
root

Node(depth=0,wins=24757.5,playouts=49082, command=None)

In [35]:
root.children

[Node(depth=1,wins=36840.0,playouts=73828, command=Command(j=1, i=0)),
 Node(depth=1,wins=15.0,playouts=136, command=Command(j=2, i=1)),
 Node(depth=1,wins=5,playouts=103, command=Command(j=2, i=0)),
 Node(depth=1,wins=15.0,playouts=136, command=Command(j=0, i=1))]

In [36]:
4*3*2*1

24

In [67]:
root.children[0].children[0].children[0].children[0]#.state


Node(depth=4,wins=5765.5,playouts=11531, command=Command(j=0, i=1))

In [68]:
4 + 4*3 + 4+3*2 + 4*3*2*1

50

In [18]:
a=np.array([])

In [19]:
type(a)

numpy.ndarray

In [7]:
def get_command():
    s=input("COMMAND> ")
    j, i = s.strip().split(' ')
    return Command(int(j), int(i))

In [8]:
def apply_command(root:Node, command)->Node:
    # Either get child node that has this command or 
    # create a new root
    for child in root.children:
        if child.command==command:
            return child
    else:
        state=root.state.applyCommand(command)
        
        # Entirely new tree: no parent, no parent command etc.
        return Node(state)

In [9]:
def pick_move(root,seconds):
    end=time.time()+seconds
    while time.time()<end:
        mcts(root)
    return best(root)
    

In [ ]:
s=State()
root = Node(s)
print(root.state)
while root.state.result==Result.INPROGRESS:
    human = get_command()
    root = apply_command(root, human)
    
    if root.state.result==Result.INPROGRESS:
        computer = pick_move(root,0.5)
        root= apply_command(root, computer)
    print(root.state)
    print("")
print(f"GAME OVER, result is {root.state.result}")


 | | 
-----
 | | 
-----
 | | 
COMMAND> 0 0
O| | 
-----
 |X| 
-----
 | | 

COMMAND> 2 2
O|X| 
-----
 |X| 
-----
 | |O

COMMAND> 2 1
O|X| 
-----
 |X| 
-----
X|O|O

COMMAND> 0 2
O|X|O
-----
 |X|X
-----
X|O|O



Command(j=0, i=1)